# Streaming Feature Engineering with Lakebase PostgreSQL

Demo: Real-time features → PostgreSQL

In [ ]:
from data_generator import TransactionDataGenerator
from feature_engineering import AdvancedFeatureEngineering
from lakebase_client import LakebaseClient
import time

In [ ]:
# Get OAuth token
token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

# Initialize Lakebase client
lakebase = LakebaseClient(
    host="your-workspace.cloud.databricks.com",
    port=5432,
    database="feature_store",
    user="token",
    password=token
)

if lakebase.test_connection():
    print("✅ Connected to Lakebase PostgreSQL")

In [ ]:
# Generate streaming data
generator = TransactionDataGenerator()
streaming_df = generator.generate_transaction_data(num_users=20, num_merchants=50, rows_per_second=10)

# Apply features
feature_engineer = AdvancedFeatureEngineering()
df_with_features = feature_engineer.apply_all_features(streaming_df)

print("✅ Features applied")

In [ ]:
# Write to Lakebase PostgreSQL
query = feature_engineer.write_features_to_lakebase(
    df=df_with_features,
    lakebase_client=lakebase,
    table_name="transaction_features"
)

print("🚀 Streaming to Lakebase PostgreSQL...")
time.sleep(60)
query.stop()
print("✅ Done")

In [ ]:
# Query features
stats = lakebase.get_table_stats()
print(f"📊 Total rows: {stats['total_rows']:,}")

recent = lakebase.read_features('SELECT * FROM transaction_features ORDER BY timestamp DESC LIMIT 10')
display(recent)